## Combine Checkpointer and Memory store :

In [1]:
#Important Libraries
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_openai import ChatOpenAI
import uuid
from dotenv import load_dotenv
import os

In [2]:
#Load API Key from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
#Initialize the LLM 
model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=api_key, temperature=0)

#Initialize In-Memory Store 
in_memory_store = InMemoryStore()

#Function to store user information across sessions
def store_user_info(state: MessagesState, config, *, store=in_memory_store):
    user_id = config["configurable"]["user_id"]
    namespace = (user_id, "memories")
    #Store user's name in the memory store
    memory_id = str(uuid.uuid4())
    user_name = state["user_name"]
    memory = {"user_name": user_name}
    #Save the memory in the store
    store.put(namespace, memory_id, memory)
    return {"messages": ["User information saved successfully."]}


#Function to retrieve user information across sessions
def retrieve_user_info(state: MessagesState, config, *, store=in_memory_store):
    user_id = config["configurable"]["user_id"]
    namespace = (user_id, "memories")
    #Retrieve stored user information
    memories = store.search(namespace)
    if memories:
        info = f"Hello {memories[-1].value['user_name']}, welcome back!"
    else: 
        info = "Hello, I don't have your information yet."
    return {"messages": [info]}


#Function to handle storing or retrieving based on user input
def call_model(state: MessagesState, config):
    last_message = state["messages"][-1].content.lower()
    if "remember my name" in last_message:
        #Store user's name in sstate and in memory store
        user_name = last_message.split("remember my name is")[-1].strip()
        state["user_name"] = user_name
        return store_user_info(state, config)
    elif "what is my name" in last_message or "what is my name" in last_message:
        #Retrieve user's name from memory store
        return retrieve_user_info(state, config)
    return {"messages": ["I can remember your name if you tell me to remember it."]}


#Build the LangGraph StateGraph(workflow)
workflow = StateGraph(MessagesState)
workflow.add_node("call_model", call_model)
workflow.add_edge(START, "call_model")
workflow.add_edge("call_model", END)

#Compile the graph with Memory Saver Checkpointer
app_with_memory = workflow.compile(
    checkpointer=MemorySaver(),
      store=in_memory_store)

#Simulate sessions
def simulate_session():
    #First session: store user name
    config = {"configurable": {"user_id": "user_123"}, "thread_id": "session_1"}
    input_message  = {"messages": [{"type" : "user", "content": "Remember my name is Amin."}]}
    for chunk in app_with_memory.stream(input_message, config=config, stream_mode="values"):
        chunk["messages"][-1].pretty_print()
    #Second session: retrieve user name
    config = {"configurable": {"user_id": "user_123"}, "thread_id": "session_2"}
    input_message  = {"messages": [{"type" : "user" , "content": "What is my name?"}]}
    for chunk in app_with_memory.stream(input_message, config=config, stream_mode="values"):
        chunk["messages"][-1].pretty_print()


#Run the simulation
simulate_session()            

================================ Human Message =================================

Remember my name is Amin.
================================ Human Message =================================

User information saved successfully.
================================ Human Message =================================

What is my name?
================================ Human Message =================================

Hello amin., welcome back!
